In [93]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
train_tape = "Z:/rppg/pure_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/pure_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [90]:
phys_net = PhysNet()
phys_net.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')

Flops per frame:54260867


In [ ]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet_PURE.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet_PURE.h5')

In [94]:
phys_net.load_weights('../weights/PhysNet_PURE.h5')

In [96]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_PURE_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_PURE_UBFC.h5')

100%|██████████| 42/42 [00:13<00:00,  3.03it/s]


{'Sliding window': {'MAE': 1.637, 'RMSE': 4.107, 'R': 0.97251},
 'Whole video': {'MAE': 1.023, 'RMSE': 1.65, 'R': 0.99627}}